## Object detection 전 box작업<br>
#### object detection을 하기 위해서는 내가 box할 좌표가 필요함
#### 때문에 opencv2를 사용해서 좌표를 얻은 후 csv파일에 image이름 좌표들로 값 저장해야함.

##### https://youbidan.tistory.com/19

In [1]:
import os
import cv2, sys
import numpy as np

from matplotlib import pyplot as plt

import pandas as pd
from collections import OrderedDict #컬럼 순서를 지정하면서 데이터 프레임을 구성

In [2]:
path = 'C:/Users/JungHyeona/Documents/deeplearning_seminar/Biometry_20200722/AC/train/label'

In [7]:
def find_edge(image):
    
    blur = cv2.GaussianBlur(image, ksize=(5, 5), sigmaX=0)
    ret, thresh = cv2.threshold(blur, 127, 255, cv2.THRESH_BINARY)
    
    # edge
    edged = cv2.Canny(blur, 10, 250)

    # edge image로 closed 찾기.
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
    closed = cv2.morphologyEx(edged, cv2.MORPH_CLOSE, kernel)

    contours, _ = cv2.findContours(closed.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    total = 0

    contours_img = cv2.drawContours(image, contours, -1, (0, 255, 0), 3)

    contours_xy = np.array(contours)
    contours_xy.shape
    
    return contours_xy

In [8]:
def coordinate(contours_xy):
    x_min, x_max, y_min, y_max = 0, 0, 0, 0
    value_x = list()
    value_y = list()

    for i in range(len(contours_xy)):
        for j in range(len(contours_xy[i])):
            value_x.append(contours_xy[i][j][0][0])
            value_y.append(contours_xy[i][j][0][1])
            
            x_min = min(value_x)
            x_max = max(value_x)
            y_min = min(value_y)
            y_max = max(value_y)
            
    # x, y, w, h 값 계산하기.
    x = x_min
    y = y_min
    w = x_max - x_min
    h = y_max - y_min
    
    '''
    print("x_min : ", x_min , " / x_max : ", x_max)
    print("x_min : ", y_min , " / x_max : ", y_max)
    print("x : ", x, " / y : ", y, " / w: ", w, " / h : ", h)
    '''
    
    #return x, y, w, h
    return x_min, y_min, x_max, y_max

In [11]:
image_name = []
X, Y, W, H = [], [], [], []
X_min, Y_min, X_max, Y_max = [], [], [], []


for img_file in os.listdir(path):
    file = os.path.join(path, img_file)

    image = cv2.imread(file)
    img_name = os.path.basename(file)
    # x, y 좌표 불러오기
    contours_xy = find_edge(image)
    
    # x, y, w, h값 가져오기.
    #x, y, w, h = coordinate(contours_xy)
    
    # x_min, y_min, x_max, y_max값 가져오기.
    x_min, y_min, x_max, y_max = coordinate(contours_xy)
    
    # csv파일에 저장하기.
    #create_dataframe(img_name, x, y, w, h)

    image_name.append(img_name)
    '''
    X.append(x)
    Y.append(y)
    W.append(w)
    H.append(h)
    '''
    X_min.append(x_min)
    Y_min.append(y_min)
    X_max.append(x_max)
    Y_max.append(y_max)

# DataFrame

In [36]:
# object box의 꼭짓점 좌표들. 이미지에서 객체가 있는 곳을 자르기 위한 값.
data = {'image_name' : image_name, 'X' : X, 'Y' : Y, 'W' : W, 'H' : H}
df = pd.DataFrame(data)

In [ ]:
df.head()

In [ ]:
df.to_csv('C:/Users/JungHyeona/Documents/deeplearning_seminar/Biometry_20200722/AC/train/labelCoordinate_01.csv', index=None) #csv파일로 생성

In [12]:
# 물체가 가지는 x, y좌표값의 각 최대값과 최솟값을 의미한다.
data = {'image_name' : image_name, 'X_min' : X_min, 'Y_min' : Y_min, 'X_max' : X_max, 'Y_max' : Y_max}
df = pd.DataFrame(data)

In [14]:
df.head()

,image_name,X_min,Y_min,X_max,Y_max
0,20151103_E0000056_I0004613.png,343,111,605,383
1,20151103_E0000057_I0004695.png,453,90,698,335
2,20151103_E0000057_I0004696.png,444,81,677,326
3,20151103_E0000058_I0004790.png,402,235,578,418
4,20151103_E0000059_I0004888.png,401,161,599,359


In [15]:
df.to_csv('C:/Users/JungHyeona/Documents/deeplearning_seminar/Biometry_20200722/AC/train/labelCoordinate_minmax_01.csv', index=None) #csv파일로 생성